<a href="https://colab.research.google.com/github/mahdi943/classification-method-leaf-dataset./blob/master/CS528_Mahdi_AliPour_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [84.8 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.laun

#Loading dataset ...

In [3]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import UserDefinedFunction
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, TrainValidationSplit,ParamGridBuilder
from pyspark.ml.feature import VectorAssembler,StringIndexer, PCA
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier,MultilayerPerceptronClassifier, RandomForestClassifier


spark = SparkSession.builder.getOrCreate()
leafDF = spark.read.option('header','false').option('inferSchema','true').csv('/content/drive/MyDrive/Colab-Notebooks/Big-data/HW3/leaf.csv')
leafDF.show(5)

# I dropped index column and applied VectorAssembler for column 2-16
leafvec =  VectorAssembler(inputCols=leafDF.columns[2:],outputCol='features')
leafDF = leafvec.transform(leafDF)
leafDF = leafDF.select('features','_c0')
leafDF = leafDF.withColumnRenamed("_c0","label")
leafDF.show(10)


+---+---+-------+------+-------+-------+-------+-------+---------+---------+---------+--------+---------+---------+---------+-------+
|_c0|_c1|    _c2|   _c3|    _c4|    _c5|    _c6|    _c7|      _c8|      _c9|     _c10|    _c11|     _c12|     _c13|     _c14|   _c15|
+---+---+-------+------+-------+-------+-------+-------+---------+---------+---------+--------+---------+---------+---------+-------+
|  1|  1|0.72694|1.4742|0.32396|0.98535|    1.0|0.83592|0.0046566|0.0039465|  0.04779| 0.12795| 0.016108|0.0052323|2.7477E-4| 1.1756|
|  1|  2|0.74173|1.5257|0.36116|0.98152|0.99825|0.79867|0.0052423|0.0050016|  0.02416|0.090476|0.0081195| 0.002708|7.4846E-5|0.69659|
|  1|  3|0.76722|1.5725|0.38998|0.97755|    1.0|0.80812|0.0074573| 0.010121| 0.011897|0.057445|0.0032891|9.2068E-4|3.7886E-5|0.44348|
|  1|  4|0.73797|1.4597|0.35376|0.97566|    1.0|0.81697|0.0068768|0.0086068|  0.01595|0.065491|0.0042707|0.0011544|6.6272E-5|0.58785|
|  1|  5|0.82301|1.7707|0.44462|0.97698|    1.0|0.75493| 0.007

# Spliting data
I am trying 3 different methods:
#### 1. Decision Tree
#### 2. Multilayer Perceptron
#### 3. Random Forest

In [4]:
# Spliting dataset samely for 3 algorithms
trainDF, testDF = leafDF.randomSplit([0.75,0.25],seed=1234)

# Decision Tree Classifier

In [5]:
trainDF, testDF = leafDF.randomSplit([0.8,0.2],seed=12345)

classifierDT = DecisionTreeClassifier()

# Adding MaxDepth and MaxBins parameters to decision tree using addGrid method
myParams = (ParamGridBuilder()
  .addGrid(classifierDT.maxBins, [10,12,14])
  .addGrid(classifierDT.maxDepth,[6,9,12,14])
  .build())

# Create cross validation object
eva = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label",metricName='accuracy')
validator = CrossValidator(
    numFolds=5, 
    estimator=classifierDT,
    estimatorParamMaps=myParams,
    evaluator=eva)

modelDT = validator.fit(trainDF)
resultDF = modelDT.transform(testDF)
resultDecTree = eva.evaluate(resultDF)

print("Max Depth : ",modelDT.bestModel._java_obj.getMaxDepth())
print("Max Bins : ",modelDT.bestModel.getMaxBins())
print("Decision Tree Classifier Accuracy : ",resultDecTree)


Max Depth :  14
Max Bins :  10
Decision Tree Classifier Accuracy :  0.6949152542372882


#Multilayer Perceptron Classifier

In [6]:
trainDF, testDF = leafDF.randomSplit([0.8,0.2],seed=1234)


mlpClassifier= MultilayerPerceptronClassifier()
eva = MulticlassClassificationEvaluator(metricName='accuracy')

myParams = ParamGridBuilder().addGrid(mlpClassifier.layers, [[14,100,40],[14,300,40]]).build()

validator = CrossValidator(estimator=mlpClassifier,
                                 estimatorParamMaps=myParams,
                                 evaluator=eva,
                                 parallelism=4,
                                 numFolds=10
                                )

modelMLP = validator.fit(trainDF)

print("Layers : ",modelMLP.bestModel.getLayers())
print("BlockSize : ",modelMLP.bestModel.getBlockSize())
print("MaxIter : ",modelMLP.bestModel.getMaxIter())

resultMLP = modelMLP.transform(testDF)
accuracyMLP = eva.evaluate(resultMLP)
print("Accuracy : ", accuracyMLP)

Layers :  [14, 300, 40]
BlockSize :  128
MaxIter :  100
Accuracy :  0.64


The best accuracy that is achived is 64%. I run the code with different values of Layers, split, seed and numFold

# Random forest

In [8]:
trainDF, testDF = leafDF.randomSplit([0.85,0.15],seed=1234)

classifierRF = RandomForestClassifier()

myParams = ParamGridBuilder().addGrid(classifierRF.numTrees,[8,12,13,15])\
                            .addGrid(classifierRF.maxDepth,[6,10,11,12])\
                            .addGrid(classifierRF.impurity,['entropy','gini']).build()

validator = CrossValidator(estimator=classifierRF,
                                 estimatorParamMaps=myParams,
                                 evaluator=eva,
                                 parallelism=4,
                                 numFolds=10
                                )

modelRF = validator.fit(trainDF)

print("Num Trees : ",modelRF.bestModel.getNumTrees)
print("Max Depth : ",modelRF.bestModel._java_obj.getMaxDepth())
print("Impurtiy : ",modelRF.bestModel._java_obj.getImpurity())

resultRF = modelRF.transform(testDF)
accuracyRF = eva.evaluate(resultRF)
print("Accuracy : ", accuracyRF)

Num Trees :  12
Max Depth :  11
Impurtiy :  gini
Accuracy :  0.7272727272727273


# Report


In [10]:

results_df = spark.createDataFrame(
    [
        ("Desicion Tree", "Max Depth: 14, Max Bins: 10", str(round(resultDecTree,2))), 
        ("Multilayer Perceptron", "Layers: [14, 300, 40], Max Iter: 100, Block Size: 128", str(round(accuracyMLP,2))), 
        ("Random Forest", "Num Trees: 12, Max Depth: 11, Impurtiy: gini", str(round(accuracyRF,2))),
    ], 
    ('Method', 'Parameters', 'Accuracy'))
result = results_df.sort('Accuracy', ascending=False)
result.show(truncate = False)

+---------------------+-----------------------------------------------------+--------+
|Method               |Parameters                                           |Accuracy|
+---------------------+-----------------------------------------------------+--------+
|Random Forest        |Num Trees: 12, Max Depth: 11, Impurtiy: gini         |0.73    |
|Desicion Tree        |Max Depth: 14, Max Bins: 10                          |0.69    |
|Multilayer Perceptron|Layers: [14, 300, 40], Max Iter: 100, Block Size: 128|0.64    |
+---------------------+-----------------------------------------------------+--------+

